In [1]:
#Import necessary Libraries
import requests
import pandas as pd
import json
import urllib
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import csv
import time
import datetime
from sqlalchemy import create_engine
import pymysql

In [2]:
#Read excel file
location=pd.read_csv(r'C:\Users\megha\Documents\Projects\DarkSky\store.csv') #105 store  IDs
#API Key
authorization_code='ffb7c6d0d32876c01496218fbd979218'
#isolate latitude and longitude
lat,long=location['LATITUDE'],location['LONGITUDE']
store_id=location['LOC_ID'] # needed to append later in file
#base url
base_url="https://api.darksky.net/forecast/ffb7c6d0d32876c01496218fbd979218/"

In [3]:
#Past date weather
#Get time for history date
from datetime import date, timedelta
dates=[]
start_date = date(2019, 12, 1)
end_date = date(2019, 12, 5)
delta = timedelta(days=1)
while start_date <= end_date:
    c=start_date.strftime("%d/%m/%Y")
    dates.append(c)
    start_date += delta
date_unix=[]
for i in dates:
    f=time.mktime(datetime.datetime.strptime(i, "%d/%m/%Y").timetuple())
    date_unix.append(f)
    
#make list of URLs    
url=''
for i,j in zip(lat,long):
    for k in date_unix:
        url +=base_url+str(i)+str(',')+str(j)+str(',')+str(int(k))+str(" ")
    url_list=url.split(" ")
    url_list.remove('')
    
#Dataframe
history=pd.DataFrame()
for url in url_list:
    response=requests.get(url)
    r=response.json()
    df=json_normalize(r['currently'])
    df['LATITUDE']=r['latitude']
    df['LONGITUDE']=r['longitude']
    history= history.append(df, ignore_index=True)
    history['date'] = pd.to_datetime(history['time'],unit='s')

History_dates=pd.merge(location, history, on=['LATITUDE', 'LONGITUDE'])
#History_dates.to_csv('History.csv')

KeyError: 'currently'

In [ ]:
# Forecast for fututre
#making a list of URLs
url_f=''
for i,j in zip(lat,long):
    url_f +=base_url+str(i)+str(',')+str(j)+str(" ")
    url_list_f=url_f.split(" ")
    url_list_f.remove('')

f=pd.DataFrame()
for url,i in zip(url_list_f,store_id):
    response=requests.get(url)
    r=response.json()
    data=json_normalize(r,['daily','data'],['latitude','longitude'])
    data['date'] = pd.to_datetime(data['time'],unit='s')
    data['store_id']=i
    f = f.append(data, ignore_index=True)
#Save as CSV   
#f.to_csv('Forecasting.csv')

In [6]:
#https://docs.sqlalchemy.org/en/13/core/engines.html
#https://pythontic.com/pandas/serialization/mysql
#moving both dataframes to MySQL database
table1='DarkSkyForecast'
table2='DarkSkyHistory'
sqlEngine = create_engine('mysql+pymysql://root:megh@n@1811@localhost/darksky', pool_recycle=3600)
dbConnection = sqlEngine.connect() 
try:
    frameh = History_dates.to_sql(table2, dbConnection, if_exists='fail');
    framef = f.to_sql(table1, dbConnection, if_exists='fail')
except ValueError as vx:
    print(vx)
except Exception as ex:   
    print(ex)
else:
    print("Table %s created successfully."%tableName);   
finally:
    dbConnection.close()

name 'total' is not defined
